In [ ]:
from importlib import reload
import pandas as pd
import time
import text_preprocessing as wpp
import country_extractor as ce
import sentiment_analyser as sa
import topic_categorizer as tc
import entity_subject_extractor as ese

Reading raw data

In [ ]:
data = pd.read_csv('../data/new_data_vdss.tsv.xz', sep='\t', compression='xz')

Delete redundant columns

In [ ]:
data['date'] = data['pubtime'].str.extract(r'(\d{4}-\d{2}-\d{2})')
data['date'] = pd.to_datetime(data['date'])
del data['pubtime']
del data['medium_code']
del data['regional']
del data['doctype']
del data['language']
del data['char_count']
del data['dateline']
del data['subhead']
del data['content_id']
del data['id']
del data['rubric']
del data['doctype_description']

Text preprocessing for article content

In [ ]:
preprocessor = wpp.TextPreprocessing()

start_time = time.time()
data['content_processed'] = data['content'].apply(lambda x: preprocessor.preprocess_text(x) if isinstance(x, str) else [])
end_time = time.time()

elapsed_time = end_time - start_time

del data['content']

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

Country extraction with runtime estimation

In [ ]:
extractor = ce.CountryExtractor()
%timeit extractor.get_country(data['content'][0])

68.1 µs ± 9.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

In [ ]:
extractor = ce.CountryExtractor()
data['countries'] = data['content_processed'].apply(lambda x: extractor.get_country(x) if isinstance(x[0], str) else [])

Sentiment analysis with runtime estimation

In [ ]:
reload(sa)
sentiment_analyser = sa.SentimentAnalyser()
%timeit sentiment_analyser.get_topic_sentiments_polarity(data['content_processed'][0])

12.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
sentiment_analyser = sa.SentimentAnalyser()

start_time = time.time()
data['sentiment'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_sentiments_polarity(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 7932.10 seconds

Subjectivity analysis with runtime estimation

In [ ]:
reload(sa)
sentiment_analyser = sa.SentimentAnalyser()
%timeit sentiment_analyser.get_topic_subjectivity(data['content_processed'][0])

16.4 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
sentiment_analyser = sa.SentimentAnalyser()

start_time = time.time()
data['subjectivity'] = data['content_processed'].apply(lambda x: sentiment_analyser.get_topic_subjectivity(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data2.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 9175.09 seconds

Categorize content with runtime estimation

In [ ]:
reload(tc)
categorizer = tc.TopicCategorizer()
%timeit data['content_processed'][:1].apply(lambda x: categorizer.categorize(x))

14 ms ± 899 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
categorizer = tc.TopicCategorizer()

start_time = time.time()
data['article_category'] = data['content_processed'].apply(lambda x: categorizer.categorize(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data5.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 6759.97 seconds

Extracting entities runtime estimation

In [ ]:
reload(ese)
entity_extractor = ese.EntityAndSubjectExtractor()
%timeit processed_data['head'][:1].apply(lambda x: entity_extractor.extract_entities(x))

4.85 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [ ]:
entity_extractor = ese.EntityAndSubjectExtractor()

start_time = time.time()
data['entities_header'] = data['head'].apply(lambda x: entity_extractor.extract_entities(x))
end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time: {:.2f} seconds".format(elapsed_time))

data.to_csv('../data/processed_data_final.tsv.xz', sep='\t', index=False, compression='xz')

Elapsed Time: 830.65 seconds

In [25]:
translate_countries = ce.CountryExtractor()
data['countries_en'] = data['countries'].apply(lambda x: translate_countries.country_translation(x))

In [27]:
data.to_csv('../data/processed_data_final.tsv.xz', sep='\t', index=False, compression='xz')

Import processed data, using eval to convert content_processed and countries to list

In [28]:
processed_data = pd.read_csv('../data/processed_data_final.tsv.xz', sep='\t', compression='xz')
#proc_data['content_processed'] = proc_data['content_processed'].apply(eval)
#proc_data['countries'] = proc_data['countries'].apply(eval)

In [ ]:
reload(ese)
people_exr = ese.EntityAndSubjectExtractor()
processed_data['people'] = processed_data['head'].apply(people_exr.extract_people)

In [ ]:
del processed_data['content_processed']
processed_data.to_csv('../data/without_content.tsv.xz', sep='\t', index=False, compression='xz')

In [ ]:
without_content = pd.read_csv('../data/without_content.tsv.xz', sep='\t', compression='xz')
without_content['countries'] = without_content['countries'].apply(eval)